In [ ]:
from pyspark.sql import SparkSession
import findspark
findspark.init()
from pyspark.sql.functions import explode,col

In [ ]:
spark = SparkSession.builder\
        .appName("rdd")\
        .getOrCreate()

spotify_df = spark.read.format("json").option("inferschema","True").load("C:\\Users\\moham\\Downloads\\spotify_raw_2025-05-07 07_27_09.593907.json")

In [ ]:
def process_album(df):
#Explode items
    explode_df = df.select(explode("items").alias("item"))
#extrating album_details
    df_al = explode_df.select(
        col("item.track.album.id").alias("album_id"),
        col("item.track.album.name").alias("album_name"),
        col("item.track.album.release_date").alias("album_release_date"),
        col("item.track.album.total_tracks").alias("album_total_track"),
        col("item.track.album.external_urls.spotify").alias("album_url")
    ).drop_duplicates(['album_id'])
    df_album = df_al.withColumn("album_release_date",to_date("album_release_date"))
    return df_album

def process_artist(df):
    explode_df = df.select(explode("items").alias("item"))
    artist_explode = explode_df.select(explode("item.track.artists").alias("artists"))
    df_artist = artist_explode.select(
        col("artists.id").alias("artist_id"),
        col("artists.name").alias("artist_name"),
        col("artists.external_urls.spotify").alias("external_url")
    ).drop_duplicates(["artist_id"])
    return df_artist

def process_song(df):
    explode_df = df.select(explode("items").alias("items"))
    df_so = explode_df.select(
        col("items.track.id").alias("song_id"),
        col("items.track.album.id").alias("album_id"),
        col("items.track.album.artists")[0]["id"].alias("artist_id"),
        col("items.track.name").alias("song_name"),
        col("items.track.popularity").alias("song_popularity"),
        col("items.track.duration_ms").alias("song_duration"),
        col("items.track.external_urls.spotify").alias("song_url"),
        col("items.added_at").alias("song_added_date")
    ).drop_duplicates(["song_id"])
    df_song = df_so.withColumn("song_added_date",df_so.song_added_date.cast("date"))
    return df_song
    
    


In [ ]:
album_df = process_album(spotify_df)
artist_df = process_artist(spotify_df)
song_df = process_song(spotify_df)